In [1]:
# Boilerplate imports.
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import PIL.Image
from matplotlib import pylab as P
import matplotlib as plt1
import pickle
import matplotlib.pyplot as plt
import math
import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '0'
# import saliency2.core as saliency
import time
from skimage import segmentation
from skimage.morphology import dilation
from skimage.morphology import disk
from skimage.transform import resize
from itertools import combinations
import copy
import scipy

%matplotlib inline

# Load models

In [2]:
# def load_model(model_name):
#     if model_name=='VGG16':
#         m2 = tf.keras.applications.vgg16.VGG16(weights='imagenet', include_top=True)
#         model2 = tf.keras.models.Model([m2.inputs], m2.output)
#         return model2
#     if model_name=='InceptionV3':
#         m2 = tf.keras.applications.inception_v3.InceptionV3(weights='imagenet', include_top=True)
#         model2 = tf.keras.models.Model([m2.inputs], m2.output)
#         return model2
#     if model_name=='ResNet50':
#         m2 = tf.keras.applications.resnet50.ResNet50(include_top=True,weights='imagenet')
#         model2 = tf.keras.models.Model([m2.inputs], m2.output)
#         return model2


# # model_to_use='ResNet50'
# # model_in_use=load_model(model_to_use)
# # model_in_use.summary()

In [3]:
def load_pre_softmax_model(m2):
    r=m2.layers[-2].output
    # new_m=tf.keras.backend.function([m2.inputs],r)
    # new_m=tf.keras.models.Model([m2.inputs],r)
    new_m = tf.keras.layers.Dense(units=m2.layers[-1].units)(r)
    new_m=tf.keras.models.Model([m2.inputs],new_m)
    new_m.layers[-1].set_weights(m2.layers[-1].get_weights())
    # new_m.summary()
    return new_m

In [4]:
def load_model(model_name):
    if model_name=='VGG16':
        m2 = tf.keras.applications.vgg16.VGG16(weights='imagenet', include_top=True)
        # print(m2.layers[-1].activation)
        # m2.summary()
        model2 = tf.keras.models.Model([m2.inputs], m2.output)
        model3=load_pre_softmax_model(m2)
        return model2,model3
    if model_name=='InceptionV3':
        m2 = tf.keras.applications.inception_v3.InceptionV3(weights='imagenet', include_top=True)
        model2 = tf.keras.models.Model([m2.inputs], m2.output)
        model3=load_pre_softmax_model(m2)
        return model2,model3
    if model_name=='ResNet50':
        m2 = tf.keras.applications.resnet50.ResNet50(include_top=True,weights='imagenet')
        model2 = tf.keras.models.Model([m2.inputs], m2.output)
        model3=load_pre_softmax_model(m2)
        return model2,model3


# model_to_use='ResNet50'
# model_in_use=load_model(model_to_use)
# model_in_use.summary()

In [5]:
# m2 = tf.keras.applications.vgg16.VGG16(weights='imagenet', include_top=True)
# model2 = tf.keras.models.Model([m2.inputs], m2.output)

In [6]:
# model = tf.keras.Sequential([
#     hub.KerasLayer(
#         name='inception_v1',
#         handle='https://tfhub.dev/google/imagenet/inception_v1/classification/4',
#         trainable=False),
# ])
# model.build([None, 224, 224, 3])
# model.summary()


# def load_imagenet_labels(file_path):
#   labels_file = tf.keras.utils.get_file('ImageNetLabels.txt', file_path)
#   with open(labels_file) as reader:
#     f = reader.read()
#     labels = f.splitlines()
#   return np.array(labels)
# imagenet_labels = load_imagenet_labels('https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt')


# def top_k_predictions(img, k=5):
#   image_batch = tf.expand_dims(img, 0)
#   predictions = model(image_batch)
#   probs = tf.nn.softmax(predictions, axis=-1)
#   top_probs, top_idxs = tf.math.top_k(input=probs, k=k)
#   top_labels = imagenet_labels[tuple(top_idxs)]
#   return top_labels, top_probs[0]


# pred_label, pred_prob = top_k_predictions(im)
# for label, prob in zip(pred_label, pred_prob):
#     print(f'{label}: {prob:0.1%}')

# segment anything model

In [ ]:

from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

sam_checkpoint = "./segment-anything-main/sam_vit_h_4b8939.pth"
model_type = "vit_h"

# device = "mps"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
# sam.to(device=device)

# mask_generator = SamAutomaticMaskGenerator(sam)
mask_generator_2 = SamAutomaticMaskGenerator(
    model=sam,
    points_per_side=32,
    pred_iou_thresh=0.86,
    stability_score_thresh=0.92,
    crop_n_layers=1,
    crop_n_points_downscale_factor=2,
    min_mask_region_area=100,  # Requires open-cv to run post-processing
)

def seg_any(image):
    masks_2 = mask_generator_2.generate(image)
    masks_new=[i['segmentation'] for i in masks_2]
    return masks_new

# segmenting

In [ ]:
def generate_segments(image,pth,redo=False):
    if os.path.exists(pth+'_segs'+'.pkl') and redo==False :
        with open(pth+'_segs'+'.pkl', 'rb') as file:
            segs=pickle.load(file)
        print('loaded segments from file')
        return segs

    else:
        print('started segmentation')
        start_time = time.time()
        segs=seg_any(image)
        # segs=_get_segments_felzenszwalb(im,min_size=50) #change here if there is a change in shape
        #segs=_get_segments_felzenszwalb(im,scale_list=[10],min_size=10) #change here if there is a change in shape
        # segs=saliency.xrai._get_segments_felzenszwalb(im,scale_list=[10],min_size=10) #change here if there is a change in shape
        # segs=get_masks()
        end_time = time.time()
        print('time taken by segmentation : '+str(end_time-start_time)+' sec')
        print('completed segmentation')
        with open(pth+'_segs'+'.pkl', 'wb') as file:
            pickle.dump(segs, file)
            file.close()

        return segs


def make_segs_disjoint(segs):
    x=segs[0]*1
    for i in range(1,len(segs)):
        x=x+(segs[i]*(1.01**i))
    new_segs=[]
    for k in np.unique(x):
        new_segs.append(x==k)
    return new_segs



# Image Loader

In [7]:
def input_size(model_to_use):
    if model_to_use=='VGG16' or model_to_use=='ResNet50':
        return 224,224
    if model_to_use=='InceptionV3':
        return 299,299
    
    
def image_load(model_to_use,file_path):
    print('*******************************************************************')
    fig_name=file_path.split('/')[-1]
    print(fig_name)
    
    image = PIL.Image.open(file_path)
    # image = image.resize((224,224))
    target_width, target_height = input_size(model_to_use)
    image = resize_with_pad(image, target_width, target_height, padding_color=(0, 0, 0))
    image=np.asarray(image)

    # image = tf.io.read_file(file_path)
    # image = tf.io.decode_jpeg(image, channels=3)
    # image = tf.image.resize_with_pad(image, target_height=224, target_width=224)
    # image=np.array(image)
    
    return (fig_name,image)


def resize_with_pad(image, target_width, target_height, padding_color=(0, 0, 0)):
    # Calculate the new size preserving the aspect ratio
    original_width, original_height = image.size
    aspect_ratio = original_width / original_height
    
    if target_width / target_height > aspect_ratio:
        # Width is the constraining dimension
        new_height = target_height
        new_width = int(aspect_ratio * new_height)
    else:
        # Height is the constraining dimension
        new_width = target_width
        new_height = int(new_width / aspect_ratio)
    
    # Resize the image to the new size
    resized_image = image.resize((new_width, new_height), PIL.Image.BILINEAR)
    
    # Create a new image with the target size and the padding color
    new_image = PIL.Image.new("RGB", (target_width, target_height), padding_color)
    
    # Calculate the position to paste the resized image onto the new image
    paste_position = (
        (target_width - new_width) // 2,
        (target_height - new_height) // 2
    )
    
    # Paste the resized image onto the new image
    new_image.paste(resized_image, paste_position)
    
    return new_image

# creating required folders

In [8]:
# def create_folders(pth):
#     if not os.path.exists(pth):
#         os.system('mkdir '+pth)
#     all_folders=['xrai_attributions/','xrai_predarrays/','xrai_intermediate_images/','xrai_graphs/']
#     for f in all_folders:
#         if not os.path.exists(pth+f):
#             os.system('mkdir '+pth+f)
#     for f in all_folders[1:]:
#         if not os.path.exists(pth+f+'normal/'):
#             os.system('mkdir '+pth+f+'normal/')
#         if not os.path.exists(pth+f+'complement/'):
#             os.system('mkdir '+pth+f+'complement/')

# Preprocess and predict Image with respect to model

In [9]:
def PreprocessImage(model_in_use,im):
    if model_in_use=='VGG16':
        im=tf.keras.applications.vgg16.preprocess_input(im)
        return im
    if model_in_use=='InceptionV3':
        im=tf.keras.applications.inception_v3.preprocess_input(im)
        return im
    if model_in_use=='ResNet50':
        im=tf.keras.applications.resnet50.preprocess_input(im)
        return im

def predict_img(model_in_use,im,print_data=True):
    model_in_use=model_in_use
    predictions = model_in_use(np.array([im]))
    prediction_class = np.argmax(predictions[0])
    # call_model_args = {class_idx_str: prediction_class}
    top10list=np.flip(np.array(predictions[0]).argsort())[:10]
    
    if print_data == True:
        print("Prediction class: " + str(prediction_class))
    decoded_predictions = tf.keras.applications.vgg16.decode_predictions(np.array(predictions), top=1000)[0]  # Top 5 predictions
    if print_data == True:
        for i, (imagenet_id, label, score) in enumerate(decoded_predictions):
            if i>=20:
                break
            print(f"{i + 1}:{imagenet_id} {label} ({score:.10f})")
    
    id_to_name=[0 for xyz in range(1000)]
    top1000list=np.flip(np.array(predictions[0]).argsort())
    for xyz in range(1000):
        id_to_name[top1000list[xyz]]=decoded_predictions[xyz][1]
    return(predictions,decoded_predictions,top10list,top1000list,id_to_name)

# Displaying

In [10]:
def ShowImage(im, title='', ax=None):
  if ax is None:
    plt.figure()
  plt.axis('off')
  plt.imshow(im)
  plt.title(title)
  plt.show()

In [13]:
# #tempcell

# topk=5
# available_models=['VGG16','InceptionV3','ResNet50']
# model_to_use='VGG16'
# model_using,model_wth_pre_soft_max_using=load_model(model_to_use)
# # model_in_use.summary()
# filepath_array=[]

# # filepath_array.append('/Users/nuthan/ME/Feature-visualization-and-attribution/images/Fireboat2-old.jpg')#originalfireboat picture
# # filepath_array.append('/Users/nuthan/ME/Feature-visualization-and-attribution/images/val_set/img200/ILSVRC2012_val_00021666.JPEG')#two rhodesianridgebacks
# # filepath_array.append('/Users/nuthan/ME/Feature-visualization-and-attribution/images/val_set/img200/ILSVRC2012_val_00021666.JPEG')#two rhodesianridgebacks
# # filepath_array.append('/Users/nuthan/ME/Feature-visualization-and-attribution/images/notfireboat.jpg')#removed fireboat from picture
# # filepath_array.append('/Users/nuthan/ME/Feature-visualization-and-attribution/images/fireboat_all_varients/n03344393_8534.JPEG')#skyliner in front of fireboat
# # filepath_array.append('/Users/nuthan/ME/Feature-visualization-and-attribution/images/fireboat3.jpg')#fireboat in dark
# # filepath_array.append('/Users/nuthan/ME/Feature-visualization-and-attribution/images/fireboat_all_varients/n03344393_773.JPEG')#fireboat from validation set
# # filepath_array.append('/Users/nuthan/ME/Feature-visualization-and-attribution/images/val_set/img200/ILSVRC2012_val_00001651.JPEG')#two rhodesianridgebacks
# filepath_array.append('/Users/nuthan/ME/Feature-visualization-and-attribution/people.jpg')#two rhodesianridgebacks


# for file_path in filepath_array:
#     start_time = time.time()
#     pth='temp/'
#     (fig_name,image)=image_load(model_to_use,file_path) #todo load according to model
#     pth=pth+model_to_use+'_'+fig_name
#     # create_folders(pth) #undo this
#     # im = PreprocessImage(model_to_use,image) #todo preprocess according to model
#     # ShowImage(image,title=fig_name)
#     # (predictions,decoded_predictions,top10list,top1000list,id_to_name)=predict_img(model_using,im,print_data=True)
#     # print(top10list)
#     # break
#     segs=generate_segments(image,pth,redo=False)
#     end_time = time.time()
#     print('completed for fig_name')
#     print('time taken : '+str(end_time-start_time)+' sec')




# Pixel wise attribution --> Integrated gradients

In [14]:
#for one output
def call_model_function2(model_in_use,images,target_class_idx_list=[554]):
    target_class_idx =  target_class_idx_list[0]
    images = tf.convert_to_tensor(images)
    with tf.GradientTape() as tape:
        tape.watch(images)
        # print('starting inference')
        output_layer = model_in_use(images)
        # print('completed inference')
        output_node = output_layer[:,target_class_idx]
        # print('starting gradient calculation for the label')
        
        gradients = np.array(tape.gradient(output_node, images))

        # print('completed gradient calculation for the label')
        return  gradients

#for multiple outputs
def call_model_function(images,target_class_idx_list=[554]):
    target_class_idx =  target_class_idx_list
    images = tf.convert_to_tensor(images)
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(images)
        # print('starting inference')
        output_layer = model2(images)
        # print('completed inference')
        output_node = tf.gather(output_layer, target_class_idx, axis=1)
        # print('starting gradient calculation for the label')
        print(output_node.shape)
        gradients = []
        for i in range(len(target_class_idx)):
            gradient = np.array(tape.gradient(output_node[:,i], images))
            gradients.append(gradient)
            
        gradients=np.asarray(gradients)
        print(np.sum(gradients))

        # print('completed gradient calculation for the label')
        return  gradients

In [15]:
def Calculate_IG_attr(model_in_use,x_value,target_class_idx_list,batch_size):
    print('started IG')
    max_min_baselines=True
    x_steps=50
    target_class_idx_list=target_class_idx_list
    if max_min_baselines is True:
        x_baselines = []
        x_baselines.append(np.min(x_value) * np.ones_like(x_value))
        x_baselines.append(np.max(x_value) * np.ones_like(x_value))
    else:
        x_baselines = []
        x_baselines.append(np.zeros_like(x_value))
        x_baselines.append(np.ones_like(x_value))
    # print(x_baselines)
    # print('completed baselining')
    grads = []
    for x_baseline in x_baselines:
        # print('starting IG for Baselinex')
        x_diff = x_value - x_baseline
        total_gradients = np.zeros_like(x_value, dtype=np.float32)
        x_step_batched = []
        for alpha in np.linspace(0, 1, x_steps):
            x_step = x_baseline + alpha * x_diff
            x_step_batched.append(x_step)
            # if len(x_step_batched) == batch_size or alpha == 1:
            if alpha == 1:
                x_step_batched = np.asarray(x_step_batched)
                # print('sending to model')
                # print(x_step_batched.shape)
                
                call_model_output = call_model_function2(model_in_use,x_step_batched,target_class_idx_list=target_class_idx_list)
        
            # self.format_and_check_call_model_output(call_model_output,
            #                                         x_step_batched.shape,
            #                                         self.expected_keys)
        
                total_gradients += call_model_output.sum(axis=0)
                x_step_batched = []
        
        grads.append(total_gradients * x_diff / x_steps)
    attr = np.mean(grads, axis=0)
    print('completed IG calculation')
    attr=attr.max(axis=-1) #change here if there is a change in shape
    print('picking up max gradients of 3 channels')
    return attr

In [16]:
def check_pixelwise_attributions_and_load(model_in_use,topk,top10list,pth,redo=False):

    if os.path.exists(pth+'_pwa'+'.pkl') and redo==False:
        with open(pth+'_pwa'+'.pkl', 'rb') as file:
            pixelwise_attributions=pickle.load(file)
        print('loaded pixel wise attributions from file')
        return pixelwise_attributions

    else:
        print('calculating pixel wise attributions')
        pixelwise_attributions={}
        for i in range(topk):
            print('for '+str(top10list[i]))
            target_class_idx_list=[top10list[i]]
            pixelwise_attributions[top10list[i]]=Calculate_IG_attr(model_in_use,im,target_class_idx_list,batch_size=20)
        with open(pth+'_pwa'+'.pkl', 'wb') as file:
            pickle.dump(pixelwise_attributions, file)
            file.close()
        return pixelwise_attributions

        # dont save them into pkl file we can save along wirth xrai attribution along with segmentation

# segment wise attribution

In [17]:

def _gain_density(mask1, attr, mask2=None):
  # Compute the attr density over mask1. If mask2 is specified, compute density
  # for mask1 \ mask2
  if mask2 is None:
    added_mask = mask1
  else:
    added_mask = _get_diff_mask(mask1, mask2)
  if not np.any(added_mask):
    return -np.inf
  else:
    return attr[added_mask].mean()


def _get_diff_mask(add_mask, base_mask):
  return np.logical_and(add_mask, np.logical_not(base_mask))


def _get_diff_cnt(add_mask, base_mask):
  return np.sum(_get_diff_mask(add_mask, base_mask))

def _unpack_segs_to_masks(segs):
  masks = []
  for seg in segs:
    for l in range(seg.min(), seg.max() + 1):
      masks.append(seg == l)
  return masks

def _normalize_image(im, value_range, resize_shape=None):
  """Normalize an image by resizing it and rescaling its values.

  Args:
      im: Input image.
      value_range: [min_value, max_value]
      resize_shape: New image shape. Defaults to None.

  Returns:
      Resized and rescaled image.
  """
  im_max = np.max(im)
  im_min = np.min(im)
  im = (im - im_min) / (im_max - im_min)
  im = im * (value_range[1] - value_range[0]) + value_range[0]
  if resize_shape is not None:
    im = resize(im,
                resize_shape,
                order=3,
                mode='constant',
                preserve_range=True,
                anti_aliasing=True)
  return im

In [18]:
def calculate_XRAI_attr(attr,segs):
    output_attr = -np.inf * np.ones(shape=attr.shape, dtype=float)
    list_of_masks_for_prediction=[]
    n_masks = len(segs)
    current_area_perc = 0.0
    area_perc_th=1.0
    min_pixel_diff=5
    current_mask = np.zeros(attr.shape, dtype=bool)
    
    masks_trace = []
    remaining_masks = {ind: mask for ind, mask in enumerate(segs)}
    
    added_masks_cnt = 1
    # While the mask area is less than area_th and remaining_masks is not empty
    while current_area_perc <= area_perc_th:
      best_gain = -np.inf
      best_key = None
      remove_key_queue = []
      for mask_key in remaining_masks:
        mask = remaining_masks[mask_key]
        # If mask does not add more than min_pixel_diff to current mask, remove
        mask_pixel_diff = _get_diff_cnt(mask, current_mask)
        if mask_pixel_diff < min_pixel_diff:
          remove_key_queue.append(mask_key)
          # if _logger.isEnabledFor(logging.DEBUG):
          #   _logger.debug('Skipping mask with pixel difference: {:.3g},'.format(
          #       mask_pixel_diff))
          continue
        gain = _gain_density(mask, attr, mask2=current_mask)
        if gain > best_gain:
          best_gain = gain
          best_key = mask_key
      for key in remove_key_queue:
        del remaining_masks[key]
      if not remaining_masks:
        break
      added_mask = remaining_masks[best_key]
      mask_diff = _get_diff_mask(added_mask, current_mask)
      masks_trace.append((mask_diff, best_gain))

      list_of_masks_for_prediction.append(best_key)
      current_mask = np.logical_or(current_mask, added_mask)
      current_area_perc = np.mean(current_mask)
      
      output_attr[mask_diff] = best_gain
      del remaining_masks[best_key]  # delete used key
      # plt.imshow(output_attr)
      # plt.show()
      # if _logger.isEnabledFor(logging.DEBUG):
      #   current_attr_sum = np.sum(attr[current_mask])
      #   _logger.debug(
      #       '{} of {} masks added,'
      #       'attr_sum: {}, area: {:.3g}/{:.3g}, {} remaining masks'.format(
      #           added_masks_cnt, n_masks, current_attr_sum, current_area_perc,
      #           area_perc_th, len(remaining_masks)))
      added_masks_cnt += 1
    
    uncomputed_mask = output_attr == -np.inf
    # Assign the uncomputed areas a value such that sum is same as ig
    output_attr[uncomputed_mask] = _gain_density(uncomputed_mask, attr)
    # masks_trace = [v[0] for v in sorted(masks_trace, key=lambda x: -x[1])]
    # if np.any(uncomputed_mask):
    #   masks_trace.append(uncomputed_mask)
    # if integer_segments:
    #   attr_ranks = np.zeros(shape=attr.shape, dtype=int)
    #   for i, mask in enumerate(masks_trace):
    #     attr_ranks[mask] = i + 1
    #   return output_attr, attr_ranks
    # else:
    #   return output_attr, masks_trace
    return output_attr,list_of_masks_for_prediction

In [19]:
def check_segmentwise_attributions_and_load(pixelwise_attributions,topk,top10list,pth,segs,redo=False):
    list_of_masks={}
    if os.path.exists(pth+'_swa'+'.pkl') and redo==False:
        with open(pth+'_swa'+'.pkl', 'rb') as file:
            (segmentwise_attributions,list_of_masks)=pickle.load(file)
        print('loaded segment wise attributions from file')
        return (segmentwise_attributions,list_of_masks)

    else:
        print('calculating segment wise attributions')
        segmentwise_attributions={}
        for i in range(topk):
            print('for '+str(top10list[i]))
            target_class_idx_list=[top10list[i]]
            segmentwise_attributions[top10list[i]],list_of_masks_for_prediction=calculate_XRAI_attr(pixelwise_attributions[top10list[i]],segs)
            # print(list_of_masks_for_prediction)
            list_of_masks[top10list[i]]=list_of_masks_for_prediction

        with open(pth+'_swa'+'.pkl', 'wb') as file:
            pickle.dump((segmentwise_attributions,list_of_masks), file)
            file.close()
        return (segmentwise_attributions,list_of_masks)

        # dont save them into pkl file we can save along wirth xrai attribution along with segmentation

# apply mask on image

In [20]:

def mask_on_image(im , mask, masking_type):
    new_pic=[]
    new_im=np.transpose(im, (2, 0, 1))
    if masking_type=='normal/':
        for i in new_im:
            new_pic.append(i*(~mask))
    else:
        for i in new_im:
            new_pic.append(i*(mask))
    new_im=np.transpose(new_pic, (1,2,0))
    return new_im

# Redaction and Prediction

## calculating predictions for each redaction -variation 1

In [21]:

def calculate_predictions_for_each_redaction(model_in_use,segmentwise_attributions,image,im,topk,top10list,prediction_id,list_of_masks,new_segs,predictions):    
    top_heatmaps=100
    masking_type='normal/'
    dict1={}
    dict2={}
    dict_for_redacted_images=[]
    all_predictions_after_this_redaction=[]
    for i in top10list:
        dict1[i]=[]
        dict2[i]=[]
        dict1[i].append(predictions[0][i].numpy())
        # dict2[i].append(100.0)
    numbered_masks = {ind: mask for ind, mask in enumerate(new_segs)}
    oldmask=np.zeros(numbered_masks[0].shape, dtype=bool)
    all_predictions_after_this_redaction.append([])
    # for i in top10list:
    #         dict1[i].append(predictions1[0][i].numpy())
    

    # while top_heatmaps>=0:
    #     # for top_heatmaps in range(100,-0.9,-0.1)
    #     mask = segmentwise_attributions[prediction_id] > np.percentile(segmentwise_attributions[prediction_id], top_heatmaps)
    #     new_im=mask_on_image(im , mask, masking_type)
    #     new_im1=mask_on_image(image , mask, masking_type)
    #     (predictions1,decoded_predictions1,top10list1,top1000list1,id_to_name1)=predict_img(model_in_use,new_im,print_data=False)
    #     dict_for_redacted_images.append(new_im1)
    #     for i in top10list:
    #         dict1[i].append(predictions1[0][i].numpy())
    #     top_heatmaps=top_heatmaps-1

    for mask_id in list_of_masks[prediction_id]:
        # for top_heatmaps in range(100,-0.9,-0.1)
        mask=numbered_masks[mask_id]
        mask=np.logical_or(mask,oldmask)
        new_im=mask_on_image(im , mask, masking_type)
        new_im1=mask_on_image(image , mask, masking_type)
        oldmask=mask
        # plt.imshow(mask)
        # plt.show()
        
        (predictions1,decoded_predictions1,top10list1,top1000list1,id_to_name1)=predict_img(model_in_use,new_im,print_data=False)
        dict_for_redacted_images.append(new_im1)
        all_predictions_after_this_redaction.append(predictions1[0].numpy())
        for i in top10list:
            dict1[i].append(predictions1[0][i].numpy())

    # plt.imshow(mask)
    # plt.show()
    
    for key in dict1:
        for k in range(len(dict1[key])):                
            dict2[key].append((dict1[key][k]/dict1[key][0])*100)
    return [dict1,dict2,dict_for_redacted_images,all_predictions_after_this_redaction]




In [22]:

def load_predictions_by_redactions(model_in_use,segmentwise_attributions,image,im,topk,top10list,pth,list_of_masks,new_segs,predictions,redo=False,dont_save=False):

    if os.path.exists(pth+'_redactions'+'.pkl') and redo==False:
        with open(pth+'_redactions'+'.pkl', 'rb') as file:
            x=pickle.load(file)
            # [final_prediction_dictionary,final_prediction_dictionary_with_percentages,final_redacted_images]=pickle.load(file)
        print('loaded final predictions after redactions from file for type1 check')
        return x
        # return final_prediction_dictionary,final_prediction_dictionary_with_percentages,final_redacted_images,all_predictions_after_redaction

    else:
        all_predictions_after_redaction={}
        final_prediction_dictionary={}
        final_prediction_dictionary_with_percentages={}
        final_redacted_images={}
        for prediction_id in top10list[:topk]:
            print('loading predictions while redating '+str(prediction_id))
            prediction_dictionary,prediction_dictionary_with_percentages,dict_for_redacted_images,all_predictions_after_this_redaction=calculate_predictions_for_each_redaction(model_in_use,segmentwise_attributions,image,im,topk,top10list,prediction_id,list_of_masks,new_segs,predictions)
            final_prediction_dictionary[prediction_id]=prediction_dictionary
            final_prediction_dictionary_with_percentages[prediction_id]=prediction_dictionary_with_percentages
            all_predictions_after_redaction[prediction_id]=all_predictions_after_this_redaction
            # final_redacted_images[prediction_id]=dict_for_redacted_images # undo this if you are not using function to calcluate redaction images

        if dont_save== False:
            with open(pth+'_redactions'+'.pkl', 'wb') as file:
                pickle.dump([final_prediction_dictionary,final_prediction_dictionary_with_percentages,final_redacted_images,all_predictions_after_redaction], file)
                file.close()

        return [final_prediction_dictionary,final_prediction_dictionary_with_percentages,final_redacted_images,all_predictions_after_redaction]


In [23]:
def show_redacted_image(segmentwise_attributions,prediction_id,redaction1_percentile):
    masking_type='normal/'
    mask = segmentwise_attributions[prediction_id] > np.percentile(segmentwise_attributions[prediction_id], 100-redaction1_percentile)
    new_im1=mask_on_image(image , mask, masking_type)
    return new_im1,mask
    

# Type-1 and Type-2 check

## plotting

In [24]:

def plot_for_type1_redaction1(redacted_image_with_pred1,redacted_mask_with_pred1,final_prediction_dictionary,first_redaction,id_to_name,prediction1,prediction2,redaction1_percentile,type_of_graph):
    ROWS=1
    COLS=2
    UPSCALE_FACTOR = 10
    fig, axs = plt.subplots(ROWS, COLS, figsize=(ROWS * UPSCALE_FACTOR, COLS * (UPSCALE_FACTOR-5)))
    axs.flat[0].imshow(redacted_image_with_pred1)
    axs.flat[1].imshow(redacted_mask_with_pred1)
    
    axs.flat[0].axis('off')
    axs.flat[1].axis('off')
    plt.show()

    ROWS=1
    COLS=2
    UPSCALE_FACTOR = 10
    fig, axs = plt.subplots(ROWS, COLS, figsize=(ROWS * UPSCALE_FACTOR, COLS * (UPSCALE_FACTOR-8)))
    # for xy in list(final_prediction_dictionary[prediction1].keys())[:5]:
    #     axs.flat[0].plot(range(len(final_prediction_dictionary[prediction1][xy])),final_prediction_dictionary[prediction1][xy],label=id_to_name[xy])
    axs.flat[0].plot(range(len(final_prediction_dictionary[prediction1][prediction1])),final_prediction_dictionary[prediction1][prediction1],label=id_to_name[prediction1])
    axs.flat[0].plot(range(len(final_prediction_dictionary[prediction1][prediction2])),final_prediction_dictionary[prediction1][prediction2],label=id_to_name[prediction2])
    axs.flat[0].legend()
    if type_of_graph == 'prob':
        axs.flat[0].set_title('probabilities vs redactions')
        axs.flat[0].set_ylabel('probabilities')
        axs.flat[0].set_xlabel('redactions')
    elif type_of_graph == 'pre_sm':
        axs.flat[0].set_title('pre_softmax vs redactions')
        axs.flat[0].set_ylabel('pre_softmax')
        axs.flat[0].set_xlabel('redactions')
    axs.flat[0].axvline(x=redaction1_percentile, color='r', linestyle=':',label=str(redaction1_percentile))
    # for xy in list(first_redaction.keys())[:5]:
    #     axs.flat[1].plot(range(len(first_redaction[xy])),first_redaction[xy],label=id_to_name[xy])
    axs.flat[1].plot(range(len(first_redaction[prediction1])),first_redaction[prediction1],label=id_to_name[prediction1])
    axs.flat[1].plot(range(len(first_redaction[prediction2])),first_redaction[prediction2],label=id_to_name[prediction2])
    axs.flat[1].legend()
    axs.flat[1].set_title('percentages vs redactions')
    axs.flat[1].set_ylabel('percentages')
    axs.flat[1].set_xlabel('redactions')
    axs.flat[1].axvline(x=redaction1_percentile, color='r', linestyle=':',label=str(redaction1_percentile))
    plt.show()

def plot_for_type1_redaction2(redacted_image_with_pred2,redacted_mask_with_pred2,final_prediction_dictionary,second_redaction,id_to_name,prediction1,prediction2,redaction2_percentile):


    ROWS=1
    COLS=2
    UPSCALE_FACTOR = 10
    fig, axs = plt.subplots(ROWS, COLS, figsize=(ROWS * UPSCALE_FACTOR, COLS * (UPSCALE_FACTOR-5)))
    axs.flat[0].imshow(redacted_image_with_pred2)
    axs.flat[1].imshow(redacted_mask_with_pred2)
    axs.flat[0].axis('off')
    axs.flat[1].axis('off')
    plt.show()


    ROWS=1
    COLS=2
    UPSCALE_FACTOR = 10
    fig, axs = plt.subplots(ROWS, COLS, figsize=(ROWS * UPSCALE_FACTOR, COLS * (UPSCALE_FACTOR-8)))
    # for xy in list(final_prediction_dictionary[prediction2].keys())[:5]:
    #     axs.flat[0].plot(range(len(final_prediction_dictionary[prediction2][xy])),final_prediction_dictionary[prediction2][xy],label=id_to_name[xy])

    axs.flat[0].plot(range(len(final_prediction_dictionary[prediction2][prediction1])),final_prediction_dictionary[prediction2][prediction1],label=id_to_name[prediction1])
    axs.flat[0].plot(range(len(final_prediction_dictionary[prediction2][prediction2])),final_prediction_dictionary[prediction2][prediction2],label=id_to_name[prediction2])
    axs.flat[0].legend()
    axs.flat[0].set_title('probabilities vs redaction percentile')
    axs.flat[0].set_ylabel('probabilities')
    axs.flat[0].set_xlabel('redaction percentile')
    axs.flat[0].axvline(x=redaction2_percentile, color='r', linestyle=':',label=str(redaction2_percentile))
    # for xy in list(second_redaction.keys())[:5]:
    #     axs.flat[1].plot(range(len(second_redaction[xy])),second_redaction[xy],label=id_to_name[xy])

    axs.flat[1].plot(range(len(second_redaction[prediction1])),second_redaction[prediction1],label=id_to_name[prediction1])
    axs.flat[1].plot(range(len(second_redaction[prediction2])),second_redaction[prediction2],label=id_to_name[prediction2])
    axs.flat[1].legend()
    axs.flat[1].set_title('percentages vs redaction percentile')
    axs.flat[1].set_ylabel('percentages')
    axs.flat[1].set_xlabel('redaction percentile')
    axs.flat[1].axvline(x=redaction2_percentile, color='r', linestyle=':',label=str(redaction2_percentile))
    plt.show()


def plot_for_type2_sequence_of_masks(segmentwise_attributions,prediction1,prediction2,redaction1_percentile,redaction2_percentile):
    xy=segmentwise_attributions[prediction1] > np.percentile(segmentwise_attributions[prediction1], 100)
    mask_temp=np.zeros(xy.shape)
    mask_temp_array=[]
    mask_temp_array2=[]
    for o in range(redaction1_percentile+1):
        mask_temp_new = segmentwise_attributions[prediction1] > np.percentile(segmentwise_attributions[prediction1], 100-o)
        if np.all(mask_temp_new== mask_temp):
            continue
        else:
            mask_temp= mask_temp_new
            mask_temp_array.append(mask_temp_new)
            mask_temp_array2.append(o)
   
    n=len(mask_temp_array)                
    # n=redaction1_percentile+1
    ROWS = math.floor(math.sqrt(n))
    COLS = math.ceil(n / ROWS)
    UPSCALE_FACTOR = 10
    fig, axs = plt.subplots(ROWS, COLS, figsize=(ROWS * UPSCALE_FACTOR, COLS * (UPSCALE_FACTOR-5)))
    for o in range(len(mask_temp_array)):
        axs.flat[o].imshow(mask_temp_array[o])
        axs.flat[o].set_title(mask_temp_array2[o])
    # for o in range(redaction1_percentile+1):
    #     mask_temp = segmentwise_attributions[prediction1] > np.percentile(segmentwise_attributions[prediction1], 100-o)
    #     axs.flat[o].imshow(mask_temp)
    plt.show()
    
    xy=segmentwise_attributions[prediction1] > np.percentile(segmentwise_attributions[prediction1], 100)
    mask_temp=np.zeros(xy.shape)
    mask_temp_array=[]
    mask_temp_array2=[]
    for o in range(redaction2_percentile+1):
        mask_temp_new = segmentwise_attributions[prediction2] > np.percentile(segmentwise_attributions[prediction2], 100-o)
        if np.all(mask_temp_new== mask_temp):
            continue
        else:
            mask_temp= mask_temp_new
            mask_temp_array.append(mask_temp_new)
            mask_temp_array2.append(o)
    
    
    n=len(mask_temp_array)                
    # n=redaction1_percentile+1
    ROWS = math.floor(math.sqrt(n))
    COLS = math.ceil(n / ROWS)
    UPSCALE_FACTOR = 10
    fig, axs = plt.subplots(ROWS, COLS, figsize=(ROWS * UPSCALE_FACTOR, COLS * (UPSCALE_FACTOR-5)))
    for o in range(len(mask_temp_array)):
        axs.flat[o].imshow(mask_temp_array[o])
        axs.flat[o].set_title(mask_temp_array2[o])
    # for o in range(redaction1_percentile+1):
    #     mask_temp = segmentwise_attributions[prediction1] > np.percentile(segmentwise_attributions[prediction1], 100-o)
    #     axs.flat[o].imshow(mask_temp)
    plt.show()

    


In [25]:
#generate mask using list of all mask ids
def get_mask(mask_id_intersection,new_segs):
    numbered_masks = {ind: mask for ind, mask in enumerate(new_segs)}
    oldmask=np.zeros(numbered_masks[0].shape, dtype=bool)
    for i in mask_id_intersection:
        mask=np.logical_or(numbered_masks[i],oldmask)
        # plt.imshow(mask)
        # plt.show()
        oldmask=mask
    return mask

In [26]:
#check if not required


def check_for_type_1_and_2_using_rankings_exp1(segs,pixelwise_attributions,segmentwise_attributions,final_prediction_dictionary,final_prediction_dictionary_with_percentages,final_redacted_images,model_in_use,im,image,topk,top10list,delta_for_type1,checkonly=None):
    combs=[i for i in range(topk)]
    cnt=0
    for (x1,x2) in list(combinations(combs, 2)):
        prediction1=top10list[x1]
        prediction2=top10list[x2]
        first_redaction=final_prediction_dictionary_with_percentages[prediction1]
        second_redaction=final_prediction_dictionary_with_percentages[prediction2]
        print(x1,x2)
        print(id_to_name[prediction1],id_to_name[prediction2])
        print(prediction1,prediction2)
        type1_flag=None
        type2_flag=None

        
        
        '''if checkonly=='type1' or checkonly==None:
            # checking for type1
            print('checking for type1')
            for i in range(len(first_redaction[prediction1])):
                flag1=None
                if first_redaction[prediction1][i] <= 100-delta_for_type1 and first_redaction[prediction2][i] >= delta_for_type1:
                    flag1=True
                    redaction1_percentile=i
                    break
        
            for i in range(len(first_redaction[prediction1])):
                flag2=None    
                if second_redaction[prediction2][i] <= 100-delta_for_type1 and second_redaction[prediction1][i] >= delta_for_type1:
                    flag2=True
                    redaction2_percentile=i
                    break
        
            if flag1==True and flag2==True:
                type1_flag=True
                cnt=cnt+1
                print('Yes its type1')
                redacted_image_with_pred1,redacted_mask_with_pred1=show_redacted_image(segmentwise_attributions,prediction1,redaction1_percentile)
                redacted_image_with_pred2,redacted_mask_with_pred2=show_redacted_image(segmentwise_attributions,prediction2,redaction2_percentile)
        
                
                print('redaction for '+str(prediction1)+' percentile at ',redaction1_percentile)
                print(' Initial predcition of '+str(prediction1),final_prediction_dictionary[prediction1][prediction1][0])
                print(' final predcition of '+str(prediction1),final_prediction_dictionary[prediction1][prediction1][redaction1_percentile])
                print(' final percentage of '+str(prediction1),first_redaction[prediction1][redaction1_percentile])
                print(' Initial predcition of '+str(prediction2),final_prediction_dictionary[prediction1][prediction2][0])
                print(' final predcition of '+str(prediction2),final_prediction_dictionary[prediction1][prediction2][redaction1_percentile])
                print(' final percentage of '+str(prediction2),first_redaction[prediction2][redaction1_percentile])
    
    
                
                plot_for_type1_redaction1(redacted_image_with_pred1,redacted_mask_with_pred1,final_prediction_dictionary,first_redaction,id_to_name,prediction1,prediction2,redaction1_percentile)
    
                # print(len(first_redaction[prediction1]))
                
                print('redaction for '+str(prediction2)+' percentile at ',redaction2_percentile)
                print(' Initial predcition of '+str(prediction2),final_prediction_dictionary[prediction2][prediction2][0])
                print(' final predcition of '+str(prediction2),final_prediction_dictionary[prediction2][prediction2][redaction2_percentile])
                print(' final percentage of '+str(prediction2),second_redaction[prediction2][redaction2_percentile])
                print(' Initial predcition of '+str(prediction1),final_prediction_dictionary[prediction2][prediction1][0])
                print(' final predcition of '+str(prediction1),final_prediction_dictionary[prediction2][prediction1][redaction2_percentile])
                print(' final percentage of '+str(prediction1),second_redaction[prediction1][redaction2_percentile])
    
                
                plot_for_type1_redaction2(redacted_image_with_pred2,redacted_mask_with_pred2,final_prediction_dictionary,second_redaction,id_to_name,prediction1,prediction2,redaction2_percentile)
    
                
                mask_for_redaction1 = segmentwise_attributions[prediction1] > np.percentile(segmentwise_attributions[prediction1], 100-redaction1_percentile)
                mask_for_redaction2 = segmentwise_attributions[prediction2] > np.percentile(segmentwise_attributions[prediction2], 100-redaction2_percentile)
        
                
                
                # print(id_to_name[prediction1])
                # plt.imshow(redacted_image_with_pred1)
                # plt.show()
                # print(id_to_name[prediction2])
                # plt.imshow(redacted_image_with_pred2)
                # plt.show()'''


        if checkonly=='type2' or checkonly==None and type1_flag!=True:
            #checking for type2
            print('checking for type2')
            # print(first_redaction[prediction1])
            # print(first_redaction[prediction2])
            for i in range(len(first_redaction[prediction1])):
                flag1=None
                # print(i,first_redaction[prediction1][i],first_redaction[prediction2][i])
                if first_redaction[prediction1][i] <= 20 and first_redaction[prediction2][i] <= 20:
                    flag1=True
                    redaction1_percentile=i
                    break
        
            for i in range(len(first_redaction[prediction1])):
                flag2=None
                # print(i,second_redaction[prediction2][i],second_redaction[prediction1][i])
        
                if second_redaction[prediction2][i] <= 20 and second_redaction[prediction1][i] <= 20:
                    flag2=True
                    redaction2_percentile=i
                    break
            # print(flag1,flag2)
            if flag1==True and flag2==True:
                
                # mask_for_redaction1 = segmentwise_attributions[prediction1] > np.percentile(segmentwise_attributions[prediction1], 100-redaction1_percentile)
                # mask_for_redaction2 = segmentwise_attributions[prediction2] > np.percentile(segmentwise_attributions[prediction2], 100-redaction2_percentile)
    
                # plt.imshow(mask_for_redaction1)
                # plt.show()
                # plt.imshow(new_im1)
                # plt.show()
                # plt.imshow(mask_for_redaction2)
                # plt.show()
                # plt.imshow(new_im1)
                # plt.show()
                mask_union=np.logical_or(mask_for_redaction1, mask_for_redaction2) 
                mask_intersection=np.logical_and(mask_for_redaction1, mask_for_redaction2) 
                mask_only_for_1=(mask_union.astype(float)-mask_for_redaction2.astype(float)).astype(bool)
                mask_only_for_2=(mask_union.astype(float)-mask_for_redaction1.astype(float)).astype(bool)
                
                
        
                redacted_image_with_pred1,redacted_mask_with_pred1=show_redacted_image(segmentwise_attributions,prediction1,redaction1_percentile)
                redacted_image_with_pred2,redacted_mask_with_pred2=show_redacted_image(segmentwise_attributions,prediction2,redaction2_percentile)
                
                
                masking_type='normal/'
                new_im=mask_on_image(im , mask_only_for_1, masking_type)
                new_im1=mask_on_image(image , mask_only_for_1, masking_type)
                (predictions1,decoded_predictions1,top10list1,top1000list1,id_to_name1)=predict_img(model_in_use,new_im,print_data=False)
                new_prediction1_percentage_after_only1=(predictions1[0][prediction1].numpy()/final_prediction_dictionary[prediction1][prediction1][0])*100
                new_prediction2_percentage_after_only1=(predictions1[0][prediction2].numpy()/final_prediction_dictionary[prediction2][prediction2][0])*100
                
                # plt.imshow(mask_only_for_1)
                # plt.show()
                # plt.imshow(new_im1)
                # plt.show()
                masking_type='normal/'
                new_im=mask_on_image(im , mask_only_for_2, masking_type)
                new_im1=mask_on_image(image , mask_only_for_2, masking_type)
                (predictions2,decoded_predictions2,top10list2,top1000list2,id_to_name2)=predict_img(model_in_use,new_im,print_data=False)
                new_prediction1_percentage_after_only2=(predictions2[0][prediction1].numpy()/final_prediction_dictionary[prediction1][prediction1][0])*100
                new_prediction2_percentage_after_only2=(predictions2[0][prediction2].numpy()/final_prediction_dictionary[prediction2][prediction2][0])*100
                # plt.imshow(mask_only_for_2)
                # plt.show()
                
                masking_type='normal/'
                new_im=mask_on_image(im , mask_intersection, masking_type)
                new_im1=mask_on_image(image , mask_intersection, masking_type)
                (predictions3,decoded_predictions3,top10list3,top1000list3,id_to_name3)=predict_img(model_in_use,new_im,print_data=False)
                new_prediction1_percentage_after_intersection=(predictions3[0][prediction1].numpy()/final_prediction_dictionary[prediction1][prediction1][0])*100
                new_prediction2_percentage_after_intersection=(predictions3[0][prediction2].numpy()/final_prediction_dictionary[prediction2][prediction2][0])*100
                
                # if True:
                if new_prediction1_percentage_after_only1 > 70 and new_prediction2_percentage_after_only1 > 70 and new_prediction1_percentage_after_only2 > 70 and new_prediction2_percentage_after_only2 >70 :
                    print('Yes its type2')
                    type2_flag=True
                    print('redaction for '+str(prediction1)+' percentile at ',redaction1_percentile)
                    print(' Initial predcition of '+str(prediction1),final_prediction_dictionary[prediction1][prediction1][0])
                    print(' final predcition of '+str(prediction1),final_prediction_dictionary[prediction1][prediction1][redaction1_percentile])
                    print(' final percentage of '+str(prediction1),first_redaction[prediction1][redaction1_percentile])
                    print(' Initial predcition of '+str(prediction2),final_prediction_dictionary[prediction1][prediction2][0])
                    print(' final predcition of '+str(prediction2),final_prediction_dictionary[prediction1][prediction2][redaction1_percentile])
                    print(' final percentage of '+str(prediction2),first_redaction[prediction2][redaction1_percentile])
    
                    plot_for_type1_redaction1(redacted_image_with_pred1,redacted_mask_with_pred1,final_prediction_dictionary,first_redaction,id_to_name,prediction1,prediction2,redaction1_percentile)
    
                    
                    print('redaction for '+str(prediction2)+' percentile at ',redaction2_percentile)
                    print(' Initial predcition of '+str(prediction2),final_prediction_dictionary[prediction2][prediction2][0])
                    print(' final predcition of '+str(prediction2),final_prediction_dictionary[prediction2][prediction2][redaction2_percentile])
                    print(' final percentage of '+str(prediction2),second_redaction[prediction2][redaction2_percentile])
                    print(' Initial predcition of '+str(prediction1),final_prediction_dictionary[prediction2][prediction1][0])
                    print(' final predcition of '+str(prediction1),final_prediction_dictionary[prediction2][prediction1][redaction2_percentile])
                    print(' final percentage of '+str(prediction1),second_redaction[prediction1][redaction2_percentile])
    
                    plot_for_type1_redaction2(redacted_image_with_pred2,redacted_mask_with_pred2,final_prediction_dictionary,second_redaction,id_to_name,prediction1,prediction2,redaction2_percentile)
    
                    print('redaction only for '+str(prediction1)+' but not for '+str(prediction2))
                    print(' Initial predcition of '+str(prediction1),final_prediction_dictionary[prediction1][prediction1][0])
                    print(' final predcition of '+str(prediction1),predictions1[0][prediction1].numpy())
                    print(' final percentage of '+str(prediction1),new_prediction1_percentage_after_only1)
                    print(' Initial predcition of '+str(prediction2),final_prediction_dictionary[prediction2][prediction2][0])
                    print(' final predcition of '+str(prediction2),predictions1[0][prediction2].numpy())
                    print(' final percentage of '+str(prediction2),new_prediction2_percentage_after_only1)
        
                    print('redaction only for '+str(prediction2)+' but not for '+str(prediction1))
                    print(' Initial predcition of '+str(prediction2),final_prediction_dictionary[prediction2][prediction2][0])
                    print(' final predcition of '+str(prediction2),predictions2[0][prediction2].numpy())
                    print(' final percentage of '+str(prediction2),new_prediction2_percentage_after_only2)
                    print(' Initial predcition of '+str(prediction1),final_prediction_dictionary[prediction1][prediction1][0])
                    print(' final predcition of '+str(prediction1),predictions2[0][prediction1].numpy())
                    print(' final percentage of '+str(prediction1),new_prediction1_percentage_after_only2)
        
                    print('redaction of intersection of '+str(prediction1)+' and '+str(prediction2))
                    print(' Initial predcition of '+str(prediction1),final_prediction_dictionary[prediction1][prediction1][0])
                    print(' final predcition of '+str(prediction1),predictions3[0][prediction1].numpy())
                    print(' final percentage of '+str(prediction1),new_prediction1_percentage_after_intersection)
                    print(' Initial predcition of '+str(prediction2),final_prediction_dictionary[prediction2][prediction2][0])
                    print(' final predcition of '+str(prediction2),predictions3[0][prediction2].numpy())
                    print(' final percentage of '+str(prediction2),new_prediction2_percentage_after_intersection)
    
                    plot_for_type2_sequence_of_masks(segmentwise_attributions,prediction1,prediction2,redaction1_percentile,redaction2_percentile)
    
                    
                    
                # plt.imshow(mask_only_for_2)
                # plt.show()
                # plt.imshow(new_im1)
                # plt.show()
                # print(id_to_name[prediction1])
                # plt.imshow(redacted_image_with_pred1)
                # plt.show()
                # plt.imshow(mask_for_redaction1)
                # plt.show()
                # # plt.imshow(mask_only_for_1)
                # # plt.show()
                # print(id_to_name[prediction2])
                # plt.imshow(redacted_image_with_pred2)
                # plt.show()
                # plt.imshow(mask_for_redaction2)
                # plt.show()
                # # plt.imshow(mask_only_for_2)
                # # plt.show()
                # plt.imshow(mask_union)
                # plt.show()
                # plt.imshow(mask_intersection)
                # plt.show()'''
        
        # break
    if type1_flag==True and type2_flag==True:
        print('Here i am , Both are true')
    print(cnt)
    return cnt
    
    
    




In [27]:
#for disjoint experiment 
def rank_the_disjoint_segments_based_on_predictions(list_of_masks,pred1,pred2,new_segs):
    a=list_of_masks[pred1]
    b=list_of_masks[pred2]
    
    ranked_a=[]
    ranked_b=[]
    ranked_a_list=[]
    ranked_b_list=[]
    adjacent_segs={}
    
    for i in range(len(a)):
        if a[i]!=b[i]:
            if a[i] not in ranked_b:
                ranked_a.append(a[i])
                ranked_a_list.append({'mask_id':a[i],'required_for_pred':True})
            else:
                ranked_a_list.append({'mask_id':a[i],'required_for_pred':False})
            if b[i] not in ranked_a:
                ranked_b.append(b[i])
                ranked_b_list.append({'mask_id':b[i],'required_for_pred':True})
            else:
                ranked_b_list.append({'mask_id':b[i],'required_for_pred':False})
        else:
            adjacent_segs[a[i]]=get_adjacent_segments(new_segs,a[i])
            #remove these once you figure out how to check for adjacent segments
            # ranked_a.append(a[i])
            # ranked_b.append(b[i])
            ranked_a_list.append({'mask_id':a[i],'required_for_pred':'equal'})
            ranked_b_list.append({'mask_id':b[i],'required_for_pred':'equal'})
    
    
    for i in adjacent_segs.keys():
        for j in ranked_a_list:
            if i == j['mask_id'] and j['required_for_pred']=='equal':
                towards_a=0
                towards_b=0
                for k in adjacent_segs[i]:
                    if k in ranked_a:
                        towards_a=towards_a+1
                    elif k in ranked_b:
                        towards_b=towards_b+1
                if towards_a>=towards_b:
                    for kk in range(len(ranked_a_list)):
                        if ranked_a_list[kk]['mask_id']==i:
                            ranked_a_list[kk]['required_for_pred']=True
                            ranked_a_list[kk]['once_equals']=True
                    for kk in range(len(ranked_b_list)):
                        if ranked_b_list[kk]['mask_id']==i:
                            ranked_b_list[kk]['required_for_pred']=False
                            ranked_b_list[kk]['once_equals']=True
                else:
                    for kk in range(len(ranked_a_list)):
                        if ranked_a_list[kk]['mask_id']==i:
                            ranked_a_list[kk]['required_for_pred']=False
                            ranked_a_list[kk]['once_equals']=True
                    for kk in range(len(ranked_b_list)):
                        if ranked_b_list[kk]['mask_id']==i:
                            ranked_b_list[kk]['required_for_pred']=True
                            ranked_b_list[kk]['once_equals']=True
                        
                    
    # print(adjacent_segs)
    # print('********************************')        
    # print(ranked_a_list)
    # print('********************************')
    # print(ranked_b_list)
    return adjacent_segs,ranked_a_list,ranked_b_list


def get_adjacent_segments(new_segs,x):
    shape=new_segs[x].shape
    x1=new_segs[x]
    new_array=copy.deepcopy(x1)
    for i in range(shape[0]):
        for j in range(shape[0]):
            if x1[i][j]==True:
                if i+1 < shape[0] and x1[i+1][j]==False:
                    new_array[i+1][j]=True
                if i-1 >= 0 and x1[i-1][j]==False:
                    new_array[i-1][j]=True
                if j+1 < shape[0] and x1[i][j+1]==False:
                    new_array[i][j+1]=True
                if j-1 >= 0 and x1[i][j-1]==False:
                    new_array[i][j-1]=True
    x3=[]
    for i in range(len(new_segs)):
        x2=np.logical_and(new_segs[i],new_array)
        if np.any(x2)==True and i!=x:
            x3.append(i)
    return x3

    

In [28]:
#for disjoint experiment


def calculate_predictions_for_each_redaction_new(model_in_use,segmentwise_attributions,image,im,topk,top10list,new_segs,prediction1,prediction2,ranked_list,predictions):
    numbered_masks = {ind: mask for ind, mask in enumerate(new_segs)}
    masking_type='normal/'
    dict_for_redacted_images=['temp']
    dict_for_redacted_masks=['temp']
    dict1={}
    dict1[prediction1]=[]
    dict1[prediction2]=[]
    dict1[prediction1].append(predictions[0][prediction1].numpy())
    dict1[prediction2].append(predictions[0][prediction2].numpy())
    dict2={}
    dict2[prediction1]=[]
    dict2[prediction2]=[]
    oldmask=np.zeros(numbered_masks[0].shape, dtype=bool)
    redact_segments_for_this=[True,False]
    
    for k in redact_segments_for_this:
        for i in ranked_list:
            if k == i['required_for_pred']:
                mask=numbered_masks[i['mask_id']]
                mask=np.logical_or(mask,oldmask)
                new_im=mask_on_image(im , mask, masking_type)
                new_im1=mask_on_image(image , mask, masking_type)
                oldmask=mask
                (predictions1,decoded_predictions1,top10list1,top1000list1,id_to_name1)=predict_img(model_in_use,new_im,print_data=False)
                
                # dict_for_redacted_images.append(new_im1) 
                # dict_for_redacted_masks.append(mask) 
                
                dict1[prediction1].append(predictions1[0][prediction1].numpy())
                dict1[prediction2].append(predictions1[0][prediction2].numpy())

        if k == True:
            done_with_one_set=len(dict1[prediction1])
        

    
    
    # print(dict_for_redacted_images)
    for key in dict1:
        for k in range(len(dict1[key])):
            dict2[key].append((dict1[key][k]/dict1[key][0])*100)
    return dict1,dict2,dict_for_redacted_images,dict_for_redacted_masks,done_with_one_set




In [29]:
#for disjoint experiment
def compare_topk(model_in_use,segmentwise_attributions,image,im,topk,top10list,new_segs,predictions,pth,list_of_masks,redo=False):
    if os.path.exists(pth+'_pred_dict'+'.pkl') and redo==False:
        with open(pth+'_pred_dict'+'.pkl', 'rb') as file:
            prediction_dictionary=pickle.load(file)
        print('loaded final predictions after redactions from file')
        return prediction_dictionary
    
    else:
        print('starting redactions')
        combs=[i for i in range(topk)]
        cnt=0
        prediction_dictionary={}
        for (x1,x2) in list(combinations(combs, 2)):
            prediction1=top10list[x1]
            prediction2=top10list[x2]
            prediction_dictionary[f'{prediction1} vs {prediction2}']={}
            prediction_dictionary[f'{prediction1} vs {prediction2}'][f'{prediction1}']=[]
            prediction_dictionary[f'{prediction1} vs {prediction2}'][f'{prediction2}']=[]
            adjacent_segs,ranked_pred1_list,ranked_pred2_list = rank_the_disjoint_segments_based_on_predictions(list_of_masks,prediction1,prediction2,new_segs)
            prediction_dictionary1,prediction_dictionary_with_percentages1,dict_for_redacted_images1,dict_for_redacted_masks1,done_with_one_set1=calculate_predictions_for_each_redaction_new(model_in_use,segmentwise_attributions,image,im,topk,top10list,new_segs,prediction1,prediction2,ranked_pred1_list,predictions)
            prediction_dictionary2,prediction_dictionary_with_percentages2,dict_for_redacted_images2,dict_for_redacted_masks2,done_with_one_set2=calculate_predictions_for_each_redaction_new(model_in_use,segmentwise_attributions,image,im,topk,top10list,new_segs,prediction2,prediction1,ranked_pred2_list,predictions)
            # print(dict_for_redacted_images1)
            prediction_dictionary[f'{prediction1} vs {prediction2}'][f'{prediction1}'].append(prediction_dictionary1)
            prediction_dictionary[f'{prediction1} vs {prediction2}'][f'{prediction1}'].append(prediction_dictionary_with_percentages1)
            prediction_dictionary[f'{prediction1} vs {prediction2}'][f'{prediction1}'].append(dict_for_redacted_masks1)
            prediction_dictionary[f'{prediction1} vs {prediction2}'][f'{prediction1}'].append(done_with_one_set1)
            prediction_dictionary[f'{prediction1} vs {prediction2}'][f'{prediction1}'].append(ranked_pred1_list)
            
            prediction_dictionary[f'{prediction1} vs {prediction2}'][f'{prediction2}'].append(prediction_dictionary2)
            prediction_dictionary[f'{prediction1} vs {prediction2}'][f'{prediction2}'].append(prediction_dictionary_with_percentages2)
            prediction_dictionary[f'{prediction1} vs {prediction2}'][f'{prediction2}'].append(dict_for_redacted_masks2)
            prediction_dictionary[f'{prediction1} vs {prediction2}'][f'{prediction2}'].append(done_with_one_set2)
            prediction_dictionary[f'{prediction1} vs {prediction2}'][f'{prediction2}'].append(ranked_pred2_list)

            prediction_dictionary[f'{prediction1} vs {prediction2}']['adjacent_segs']=adjacent_segs
        print('completed redactions')
        with open(pth+'_pred_dict'+'.pkl', 'wb') as file:
            pickle.dump(prediction_dictionary, file)
            file.close()
        return prediction_dictionary






In [30]:
#for disjoint experiment

def plot_for_type1_redaction1_new(redacted_image_with_pred1,redacted_mask_with_pred1,first_redaction,id_to_name,prediction1,prediction2,redaction1_percentile,first_redaction_all):
    ROWS=1
    COLS=2
    UPSCALE_FACTOR = 10
    fig, axs = plt.subplots(ROWS, COLS, figsize=(ROWS * UPSCALE_FACTOR, COLS * (UPSCALE_FACTOR-5)))
    axs.flat[0].imshow(redacted_image_with_pred1)
    axs.flat[1].imshow(redacted_mask_with_pred1)
    
    axs.flat[0].axis('off')
    axs.flat[1].axis('off')
    plt.show()

    ROWS=1
    COLS=2
    UPSCALE_FACTOR = 10
    fig, axs = plt.subplots(ROWS, COLS, figsize=(ROWS * UPSCALE_FACTOR, COLS * (UPSCALE_FACTOR-8)))
    axs.flat[0].plot(range(len(first_redaction_all[0][prediction1])),first_redaction_all[0][prediction1],label=id_to_name[prediction1])
    axs.flat[0].plot(range(len(first_redaction_all[0][prediction2])),first_redaction_all[0][prediction2],label=id_to_name[prediction2])
    axs.flat[0].legend()
    axs.flat[0].set_title('probabilities vs no. of redacted segments')
    axs.flat[0].set_ylabel('probabilities')
    axs.flat[0].set_xlabel('no. of redacted segments')
    axs.flat[0].axvline(x=redaction1_percentile, color='r', linestyle=':',label=str(redaction1_percentile))
    axs.flat[0].axvline(x=first_redaction_all[3], color='b', linestyle=':',label=str(first_redaction_all[3]))
    axs.flat[1].plot(range(len(first_redaction[prediction1])),first_redaction[prediction1],label=id_to_name[prediction1])
    axs.flat[1].plot(range(len(first_redaction[prediction2])),first_redaction[prediction2],label=id_to_name[prediction2])
    axs.flat[1].legend()
    axs.flat[1].set_title('percentages vs no. of redacted segments')
    axs.flat[1].set_ylabel('percentages')
    axs.flat[1].set_xlabel('no. of redacted segments')
    axs.flat[1].axvline(x=redaction1_percentile, color='r', linestyle=':',label=str(redaction1_percentile))
    axs.flat[1].axvline(x=first_redaction_all[3], color='b', linestyle=':',label=str(first_redaction_all[3]))
    plt.show()

def plot_for_type1_redaction2_new(redacted_image_with_pred2,redacted_mask_with_pred2,second_redaction,id_to_name,prediction1,prediction2,redaction2_percentile,second_redaction_all):


    ROWS=1
    COLS=2
    UPSCALE_FACTOR = 10
    fig, axs = plt.subplots(ROWS, COLS, figsize=(ROWS * UPSCALE_FACTOR, COLS * (UPSCALE_FACTOR-5)))
    axs.flat[0].imshow(redacted_image_with_pred2)
    axs.flat[1].imshow(redacted_mask_with_pred2)
    axs.flat[0].axis('off')
    axs.flat[1].axis('off')
    plt.show()


    ROWS=1
    COLS=2
    UPSCALE_FACTOR = 10
    fig, axs = plt.subplots(ROWS, COLS, figsize=(ROWS * UPSCALE_FACTOR, COLS * (UPSCALE_FACTOR-8)))
    axs.flat[0].plot(range(len(second_redaction_all[0][prediction1])),second_redaction_all[0][prediction1],label=id_to_name[prediction1])
    axs.flat[0].plot(range(len(second_redaction_all[0][prediction2])),second_redaction_all[0][prediction2],label=id_to_name[prediction2])
    axs.flat[0].legend()
    axs.flat[0].set_title('probabilities vs no. of redacted segments')
    axs.flat[0].set_ylabel('probabilities')
    axs.flat[0].set_xlabel('no. of redacted segments')
    axs.flat[0].axvline(x=redaction2_percentile, color='r', linestyle=':',label=str(redaction2_percentile))
    axs.flat[0].axvline(x=second_redaction_all[3], color='b', linestyle=':',label=str(second_redaction_all[3]))
    axs.flat[1].plot(range(len(second_redaction[prediction1])),second_redaction[prediction1],label=id_to_name[prediction1])
    axs.flat[1].plot(range(len(second_redaction[prediction2])),second_redaction[prediction2],label=id_to_name[prediction2])
    axs.flat[1].legend()
    axs.flat[1].set_title('percentages vs no. of redacted segments')
    axs.flat[1].set_ylabel('percentages')
    axs.flat[1].set_xlabel('no. of redacted segments')
    axs.flat[1].axvline(x=redaction2_percentile, color='r', linestyle=':',label=str(redaction2_percentile))
    axs.flat[1].axvline(x=second_redaction_all[3], color='b', linestyle=':',label=str(second_redaction_all[3]))
    plt.show()


def plot_for_type2_sequence_of_masks(segmentwise_attributions,prediction1,prediction2,redaction1_percentile,redaction2_percentile):
    xy=segmentwise_attributions[prediction1] > np.percentile(segmentwise_attributions[prediction1], 100)
    mask_temp=np.zeros(xy.shape)
    mask_temp_array=[]
    mask_temp_array2=[]
    for o in range(redaction1_percentile+1):
        mask_temp_new = segmentwise_attributions[prediction1] > np.percentile(segmentwise_attributions[prediction1], 100-o)
        if np.all(mask_temp_new== mask_temp):
            continue
        else:
            mask_temp= mask_temp_new
            mask_temp_array.append(mask_temp_new)
            mask_temp_array2.append(o)
   
    n=len(mask_temp_array)                
    # n=redaction1_percentile+1
    ROWS = math.floor(math.sqrt(n))
    COLS = math.ceil(n / ROWS)
    UPSCALE_FACTOR = 10
    fig, axs = plt.subplots(ROWS, COLS, figsize=(ROWS * UPSCALE_FACTOR, COLS * (UPSCALE_FACTOR-5)))
    for o in range(len(mask_temp_array)):
        axs.flat[o].imshow(mask_temp_array[o])
        axs.flat[o].set_title(mask_temp_array2[o])
    # for o in range(redaction1_percentile+1):
    #     mask_temp = segmentwise_attributions[prediction1] > np.percentile(segmentwise_attributions[prediction1], 100-o)
    #     axs.flat[o].imshow(mask_temp)
    plt.show()
    
    xy=segmentwise_attributions[prediction1] > np.percentile(segmentwise_attributions[prediction1], 100)
    mask_temp=np.zeros(xy.shape)
    mask_temp_array=[]
    mask_temp_array2=[]
    for o in range(redaction2_percentile+1):
        mask_temp_new = segmentwise_attributions[prediction2] > np.percentile(segmentwise_attributions[prediction2], 100-o)
        if np.all(mask_temp_new== mask_temp):
            continue
        else:
            mask_temp= mask_temp_new
            mask_temp_array.append(mask_temp_new)
            mask_temp_array2.append(o)
    
    
    n=len(mask_temp_array)                
    # n=redaction1_percentile+1
    ROWS = math.floor(math.sqrt(n))
    COLS = math.ceil(n / ROWS)
    UPSCALE_FACTOR = 10
    fig, axs = plt.subplots(ROWS, COLS, figsize=(ROWS * UPSCALE_FACTOR, COLS * (UPSCALE_FACTOR-5)))
    for o in range(len(mask_temp_array)):
        axs.flat[o].imshow(mask_temp_array[o])
        axs.flat[o].set_title(mask_temp_array2[o])
    # for o in range(redaction1_percentile+1):
    #     mask_temp = segmentwise_attributions[prediction1] > np.percentile(segmentwise_attributions[prediction1], 100-o)
    #     axs.flat[o].imshow(mask_temp)
    plt.show()

    


In [31]:
#for disjoint experiment



def check_for_type_1_and_2_using_rankings_new(segs,pixelwise_attributions,segmentwise_attributions,prediction_dictionary,model_in_use,im,image,topk,top10list,delta_for_type1,checkonly=None):
    combs=[i for i in range(2)]
    cnt=0
    for (x1,x2) in list(combinations(combs, 2)):
        prediction1=top10list[x1]
        prediction2=top10list[x2]
        first_redaction=prediction_dictionary[f'{prediction1} vs {prediction2}'][f'{prediction1}'][1]
        second_redaction=prediction_dictionary[f'{prediction1} vs {prediction2}'][f'{prediction2}'][1]

        first_redaction_all=prediction_dictionary[f'{prediction1} vs {prediction2}'][f'{prediction1}']
        second_redaction_all=prediction_dictionary[f'{prediction1} vs {prediction2}'][f'{prediction2}']
        
        print(x1,x2)
        print(id_to_name[prediction1],id_to_name[prediction2])
        print(prediction1,prediction2)
        type1_flag=None
        type2_flag=None

        print('adjacent segs')
        print(prediction_dictionary[f'{prediction1} vs {prediction2}']['adjacent_segs'])
        
        if checkonly=='type1' or checkonly==None:
            # checking for type1
            print('checking for type1')
            for i in range(len(first_redaction[prediction1])):
                flag1=None
                if first_redaction[prediction1][i] <= 100-delta_for_type1 and first_redaction[prediction2][i] >= delta_for_type1:
                    flag1=True
                    redaction1_percentile=i
                    break
        
            for i in range(len(first_redaction[prediction1])):
                flag2=None    
                if second_redaction[prediction2][i] <= 100-delta_for_type1 and second_redaction[prediction1][i] >= delta_for_type1:
                    flag2=True
                    redaction2_percentile=i
                    break
        
            if flag1==True and flag2==True:
                type1_flag=True
                cnt=cnt+1
                print('Yes its type1')

                
                masking_type='normal/'
                
                print('redaction for '+str(prediction1)+' percentile at ',redaction1_percentile)
                print(' Initial predcition of '+str(prediction1),first_redaction_all[0][prediction1][0])
                print(' final predcition of '+str(prediction1),first_redaction_all[0][prediction1][redaction1_percentile])
                print(' final percentage of '+str(prediction1),first_redaction[prediction1][redaction1_percentile])
                print(' Initial predcition of '+str(prediction2),first_redaction_all[0][prediction2][0])
                print(' final predcition of '+str(prediction2),first_redaction_all[0][prediction2][redaction1_percentile])
                print(' final percentage of '+str(prediction2),first_redaction[prediction2][redaction1_percentile])
    
    
                redacted_mask_with_pred1=first_redaction_all[2][redaction1_percentile]
                redacted_image_with_pred1=mask_on_image(image , redacted_mask_with_pred1, masking_type)
                plot_for_type1_redaction1_new(redacted_image_with_pred1,redacted_mask_with_pred1,first_redaction,id_to_name,prediction1,prediction2,redaction1_percentile,first_redaction_all)
    
                # print(len(first_redaction[prediction1]))
                
                print('redaction for '+str(prediction2)+' percentile at ',redaction2_percentile)
                print(' Initial predcition of '+str(prediction2),second_redaction_all[0][prediction2][0])
                print(' final predcition of '+str(prediction2),second_redaction_all[0][prediction2][redaction2_percentile])
                print(' final percentage of '+str(prediction2),second_redaction[prediction2][redaction2_percentile])
                print(' Initial predcition of '+str(prediction1),second_redaction_all[0][prediction1][0])
                print(' final predcition of '+str(prediction1),second_redaction_all[0][prediction1][redaction2_percentile])
                print(' final percentage of '+str(prediction1),second_redaction[prediction1][redaction2_percentile])
    

                redacted_mask_with_pred2=second_redaction_all[2][redaction2_percentile]
                redacted_image_with_pred2=mask_on_image(image , redacted_mask_with_pred2, masking_type)
                plot_for_type1_redaction2_new(redacted_image_with_pred2,redacted_mask_with_pred2,second_redaction,id_to_name,prediction1,prediction2,redaction2_percentile,second_redaction_all)
    
                
                # mask_for_redaction1 = segmentwise_attributions[prediction1] > np.percentile(segmentwise_attributions[prediction1], 100-redaction1_percentile)
                # mask_for_redaction2 = segmentwise_attributions[prediction2] > np.percentile(segmentwise_attributions[prediction2], 100-redaction2_percentile)
        
                
                
                # print(id_to_name[prediction1])
                # plt.imshow(redacted_image_with_pred1)
                # plt.show()
                # print(id_to_name[prediction2])
                # plt.imshow(redacted_image_with_pred2)
                # plt.show()


        '''if checkonly=='type2' or checkonly==None and type1_flag!=True:
            #checking for type2
            print('checking for type2')
            # print(first_redaction[prediction1])
            # print(first_redaction[prediction2])
            for i in range(len(first_redaction[prediction1])):
                flag1=None
                # print(i,first_redaction[prediction1][i],first_redaction[prediction2][i])
                if first_redaction[prediction1][i] <= 20 and first_redaction[prediction2][i] <= 20:
                    flag1=True
                    redaction1_percentile=i
                    break
        
            for i in range(len(first_redaction[prediction1])):
                flag2=None
                # print(i,second_redaction[prediction2][i],second_redaction[prediction1][i])
        
                if second_redaction[prediction2][i] <= 20 and second_redaction[prediction1][i] <= 20:
                    flag2=True
                    redaction2_percentile=i
                    break
            # print(flag1,flag2)
            if flag1==True and flag2==True:
                
                # mask_for_redaction1 = segmentwise_attributions[prediction1] > np.percentile(segmentwise_attributions[prediction1], 100-redaction1_percentile)
                # mask_for_redaction2 = segmentwise_attributions[prediction2] > np.percentile(segmentwise_attributions[prediction2], 100-redaction2_percentile)
    
                # plt.imshow(mask_for_redaction1)
                # plt.show()
                # plt.imshow(new_im1)
                # plt.show()
                # plt.imshow(mask_for_redaction2)
                # plt.show()
                # plt.imshow(new_im1)
                # plt.show()
                # mask_union=np.logical_or(mask_for_redaction1, mask_for_redaction2) 
                # mask_intersection=np.logical_and(mask_for_redaction1, mask_for_redaction2) 
                # mask_only_for_1=(mask_union.astype(float)-mask_for_redaction2.astype(float)).astype(bool)
                # mask_only_for_2=(mask_union.astype(float)-mask_for_redaction1.astype(float)).astype(bool)
                
                
        
                # redacted_image_with_pred1,redacted_mask_with_pred1=show_redacted_image(segmentwise_attributions,prediction1,redaction1_percentile)
                # redacted_image_with_pred2,redacted_mask_with_pred2=show_redacted_image(segmentwise_attributions,prediction2,redaction2_percentile)
                
                
                # masking_type='normal/'
                # new_im=mask_on_image(im , mask_only_for_1, masking_type)
                # new_im1=mask_on_image(image , mask_only_for_1, masking_type)
                # (predictions1,decoded_predictions1,top10list1,top1000list1,id_to_name1)=predict_img(model_in_use,new_im,print_data=False)
                # new_prediction1_percentage_after_only1=(predictions1[0][prediction1].numpy()/final_prediction_dictionary[prediction1][prediction1][0])*100
                # new_prediction2_percentage_after_only1=(predictions1[0][prediction2].numpy()/final_prediction_dictionary[prediction2][prediction2][0])*100
                
                # # plt.imshow(mask_only_for_1)
                # # plt.show()
                # # plt.imshow(new_im1)
                # # plt.show()
                # masking_type='normal/'
                # new_im=mask_on_image(im , mask_only_for_2, masking_type)
                # new_im1=mask_on_image(image , mask_only_for_2, masking_type)
                # (predictions2,decoded_predictions2,top10list2,top1000list2,id_to_name2)=predict_img(model_in_use,new_im,print_data=False)
                # new_prediction1_percentage_after_only2=(predictions2[0][prediction1].numpy()/final_prediction_dictionary[prediction1][prediction1][0])*100
                # new_prediction2_percentage_after_only2=(predictions2[0][prediction2].numpy()/final_prediction_dictionary[prediction2][prediction2][0])*100
                # # plt.imshow(mask_only_for_2)
                # # plt.show()
                
                # masking_type='normal/'
                # new_im=mask_on_image(im , mask_intersection, masking_type)
                # new_im1=mask_on_image(image , mask_intersection, masking_type)
                # (predictions3,decoded_predictions3,top10list3,top1000list3,id_to_name3)=predict_img(model_in_use,new_im,print_data=False)
                # new_prediction1_percentage_after_intersection=(predictions3[0][prediction1].numpy()/final_prediction_dictionary[prediction1][prediction1][0])*100
                # new_prediction2_percentage_after_intersection=(predictions3[0][prediction2].numpy()/final_prediction_dictionary[prediction2][prediction2][0])*100
                
               
                
                
                
                masking_type='normal/'
                
                print('redaction for '+str(prediction1)+' percentile at ',redaction1_percentile)
                print(' Initial predcition of '+str(prediction1),first_redaction_all[0][prediction1][0])
                print(' final predcition of '+str(prediction1),first_redaction_all[0][prediction1][redaction1_percentile])
                print(' final percentage of '+str(prediction1),first_redaction[prediction1][redaction1_percentile])
                print(' Initial predcition of '+str(prediction2),first_redaction_all[0][prediction2][0])
                print(' final predcition of '+str(prediction2),first_redaction_all[0][prediction2][redaction1_percentile])
                print(' final percentage of '+str(prediction2),first_redaction[prediction2][redaction1_percentile])
    
    
                redacted_mask_with_pred1=first_redaction_all[2][redaction1_percentile]
                redacted_image_with_pred1=mask_on_image(image , redacted_mask_with_pred1, masking_type)
                plot_for_type1_redaction1_new(redacted_image_with_pred1,redacted_mask_with_pred1,first_redaction,id_to_name,prediction1,prediction2,redaction1_percentile,first_redaction_all)
    
                # print(len(first_redaction[prediction1]))
                
                print('redaction for '+str(prediction2)+' percentile at ',redaction2_percentile)
                print(' Initial predcition of '+str(prediction2),second_redaction_all[0][prediction2][0])
                print(' final predcition of '+str(prediction2),second_redaction_all[0][prediction2][redaction2_percentile])
                print(' final percentage of '+str(prediction2),second_redaction[prediction2][redaction2_percentile])
                print(' Initial predcition of '+str(prediction1),second_redaction_all[0][prediction1][0])
                print(' final predcition of '+str(prediction1),second_redaction_all[0][prediction1][redaction2_percentile])
                print(' final percentage of '+str(prediction1),second_redaction[prediction1][redaction2_percentile])
    

                redacted_mask_with_pred2=second_redaction_all[2][redaction2_percentile]
                redacted_image_with_pred2=mask_on_image(image , redacted_mask_with_pred2, masking_type)
                plot_for_type1_redaction2_new(redacted_image_with_pred2,redacted_mask_with_pred2,second_redaction,id_to_name,prediction1,prediction2,redaction2_percentile,second_redaction_all)
    

                
                
                # prediction1=top10list[1]
                # prediction2=top10list[2]
                # prediction_dictionary[f'{prediction1} vs {prediction2}']['adjacent_segs']
                mask_temp_array=prediction_dictionary[f'{prediction1} vs {prediction2}'][f'{prediction1}'][2][1:]
                n=len(mask_temp_array)                
                # n=redaction1_percentile+1
                ROWS = math.floor(math.sqrt(n))
                COLS = math.ceil(n / ROWS)
                UPSCALE_FACTOR = 10
                fig, axs = plt.subplots(ROWS, COLS, figsize=(ROWS * UPSCALE_FACTOR, COLS * (UPSCALE_FACTOR-5)))
                for o in range(len(mask_temp_array)):
                    axs.flat[o].imshow(mask_temp_array[o])
                    # axs.flat[o].set_title(mask_temp_array2[o])
                # for o in range(redaction1_percentile+1):
                #     mask_temp = segmentwise_attributions[prediction1] > np.percentile(segmentwise_attributions[prediction1], 100-o)
                #     axs.flat[o].imshow(mask_temp)
                plt.show()



                print('******************************************')
                # prediction1=top10list[1]
                # prediction2=top10list[2]
                # prediction_dictionary[f'{prediction1} vs {prediction2}']['adjacent_segs']
                temp_array=list_of_masks[prediction1]
                # mask_temp_array=prediction_dictionary[f'{prediction1} vs {prediction2}'][f'{prediction1}'][2][1:]
                n=len(temp_array)                
                # n=redaction1_percentile+1
                ROWS = math.floor(math.sqrt(n))
                COLS = math.ceil(n / ROWS)
                UPSCALE_FACTOR = 10
                fig, axs = plt.subplots(ROWS, COLS, figsize=(ROWS * UPSCALE_FACTOR, COLS * (UPSCALE_FACTOR-5)))
                for o in range(len(temp_array)):
                    axs.flat[o].imshow(new_segs[temp_array[o]])
                    axs.flat[o].set_title(f'{temp_array[o]}')
                # for o in range(redaction1_percentile+1):
                #     mask_temp = segmentwise_attributions[prediction1] > np.percentile(segmentwise_attributions[prediction1], 100-o)
                #     axs.flat[o].imshow(mask_temp)
                plt.show()
                
                print('******************************************')
                
                mask_temp_array=prediction_dictionary[f'{prediction1} vs {prediction2}'][f'{prediction2}'][2][1:]
                n=len(mask_temp_array)                
                # n=redaction1_percentile+1
                ROWS = math.floor(math.sqrt(n))
                COLS = math.ceil(n / ROWS)
                UPSCALE_FACTOR = 10
                fig, axs = plt.subplots(ROWS, COLS, figsize=(ROWS * UPSCALE_FACTOR, COLS * (UPSCALE_FACTOR-5)))
                for o in range(len(mask_temp_array)):
                    axs.flat[o].imshow(mask_temp_array[o])
                    # axs.flat[o].set_title(mask_temp_array2[o])
                # for o in range(redaction1_percentile+1):
                #     mask_temp = segmentwise_attributions[prediction1] > np.percentile(segmentwise_attributions[prediction1], 100-o)
                #     axs.flat[o].imshow(mask_temp)
                plt.show()
                


                print('******************************************')
                
                # prediction1=top10list[1]
                # prediction2=top10list[2]
                # prediction_dictionary[f'{prediction1} vs {prediction2}']['adjacent_segs']
                temp_array=list_of_masks[prediction2]
                # mask_temp_array=prediction_dictionary[f'{prediction1} vs {prediction2}'][f'{prediction1}'][2][1:]
                n=len(temp_array)                
                # n=redaction1_percentile+1
                ROWS = math.floor(math.sqrt(n))
                COLS = math.ceil(n / ROWS)
                UPSCALE_FACTOR = 10
                fig, axs = plt.subplots(ROWS, COLS, figsize=(ROWS * UPSCALE_FACTOR, COLS * (UPSCALE_FACTOR-5)))
                for o in range(len(temp_array)):
                    axs.flat[o].imshow(new_segs[temp_array[o]])
                    axs.flat[o].set_title(f'{temp_array[o]}')
                # for o in range(redaction1_percentile+1):
                #     mask_temp = segmentwise_attributions[prediction1] > np.percentile(segmentwise_attributions[prediction1], 100-o)
                #     axs.flat[o].imshow(mask_temp)
                plt.show()    
                                    


    
                # plt.imshow(mask_only_for_2)
                # plt.show()
                # plt.imshow(new_im1)
                # plt.show()
                # print(id_to_name[prediction1])
                # plt.imshow(redacted_image_with_pred1)
                # plt.show()
                # plt.imshow(mask_for_redaction1)
                # plt.show()
                # # plt.imshow(mask_only_for_1)
                # # plt.show()
                # print(id_to_name[prediction2])
                # plt.imshow(redacted_image_with_pred2)
                # plt.show()
                # plt.imshow(mask_for_redaction2)
                # plt.show()
                # # plt.imshow(mask_only_for_2)
                # # plt.show()
                # plt.imshow(mask_union)
                # plt.show()
                # plt.imshow(mask_intersection)
                # plt.show()'''
        
        break
    # if type1_flag==True and type2_flag==True:
    #     print('Here i am , Both are true')
    # print(cnt)
    return cnt
    
    
    


